In [1]:
%pip install --upgrade --quiet  langchain langchain-experimental langchain-openai neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 10.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.9/218.9 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━

In [2]:
%pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 5.3 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
#Add openAI API key
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

In [4]:
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
#Add diffbot API key
diffbot_api_key = userdata.get('diffbot_api_key')
diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=diffbot_api_key)

In [5]:
from langchain_community.graphs import Neo4jGraph
url =userdata.get('neo_url')
username = userdata.get('neo_username')
password = userdata.get('neo_pwd')
graph = Neo4jGraph(url=url, username=username, password=password)

In [6]:
!wget -nc 'https://www.yourdatateacher.com/wp-content/uploads/2021/05/Supervised-machine-learning-workflow.pdf'

--2024-01-18 07:55:35--  https://www.yourdatateacher.com/wp-content/uploads/2021/05/Supervised-machine-learning-workflow.pdf
Resolving www.yourdatateacher.com (www.yourdatateacher.com)... 89.46.109.71
Connecting to www.yourdatateacher.com (www.yourdatateacher.com)|89.46.109.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 372942 (364K) [application/pdf]
Saving to: ‘Supervised-machine-learning-workflow.pdf’

Supervised-machine- 100%[===================>] 364.20K  1.16MB/s    in 0.3s    

2024-01-18 07:55:36 (1.16 MB/s) - ‘Supervised-machine-learning-workflow.pdf’ saved [372942/372942]



In [10]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/Supervised-machine-learning-workflow.pdf")

pages = loader.load_and_split()

In [11]:

from langchain.docstore.document import Document
for i in range(0,len(pages)):
  # pages[i].page_content.replace('\n',' ')
  pages[i]=Document(page_content=pages[i].page_content.replace('\n',' '), metadata={"source": "local"})
# doc =  Document(page_content="text", metadata={"source": "local"})

In [12]:
graph_documents = diffbot_nlp.convert_to_graph_documents(pages)

In [13]:
graph.add_graph_documents(graph_documents)

In [14]:
graph.refresh_schema()

In [15]:
pages

[Document(page_content='The purpose of this e-book In this e-book, I propose the typical Supervised Machine Learning workflow that data scientists follow when they need to create a supervised model. This workflow may change from project to project, but the steps I’m going to show you in the next sections are the most common steps that are required. For each step of the workflow, I’ll give you a brief introduction and I’ll suggest one or more Python libraries that may be used in a project. www.yourdatateacher.com 2', metadata={'source': 'local'}),
 Document(page_content='Step 1: Data extraction The first step of a machine learning model is related to data. Any supervised model learns from data we feed it with. Data can be extracted from a Data Lake, a Data Warehouse, some Excel and CSV files, an SQL database or other sources. The purpose of this phase is to collect data and build the dataset we are going to work with. The dataset is a huge table made of several columns (sometimes hundre

In [16]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(temperature=0, model_name="gpt-4"),
    qa_llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),
    graph=graph,
    verbose=True,
)

In [17]:
# chain.run("What is machine learning")
chain.run("Who is Gianluca Malato?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Gianluca Malato"}) RETURN p
Full Context:
[{'p': {'name': 'Gianluca Malato', 'id': 'http://www.linkedin.com/in/gianlucamalato', 'positionHeld': 'Data Scientist'}}]

> Finished chain.


'Gianluca Malato is a Data Scientist.'